In [1]:
TEST_FILE = "glioblastoma_data/extracted_data/cell_behaviors/videos_transcriptions_csv/normal/n1/0gy/normal_[n1]_0gy_201104.nd2u2510Gy1A.csv"

DEBUG = False
VERBOSE = 0

In [2]:
!pip install drawSvg

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

import drawSvg as draw

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
def csv_radio_data_to_df(filepath):
    filename = filepath.split("/")[-1]
    print(filename)
    radiation_level = int(filename.split("_")[2].split("gy")[0])
    experiment_setting = filename.split("_")[0] + ", " + filename.split("_")[1]
    experiment_id = filename[:-4].split("_")[2]
    
    df = pd.read_csv(filepath,
                     converters={"input": lambda x: x.strip("[]").replace(' ',"").replace("'","").replace('"',"").replace(','," ").split(),
                                 "output": lambda x: x.strip("[]").replace(' ',"").replace("'","").replace('"',"").replace(','," ").split()})
    df["ancestor"] = ""
    ancestor = dict()
    start = dict()

    delete_df = False

    for i, row in df.iterrows():
        time_val = row["time"]
        event = row["event"]
        input = row["input"]
        output = row["output"]
        #print("Computing ancestor of ", time_val, event, input, output)
        if event == "begin":
            cell_id = output[0]
            ancestor[cell_id] = cell_id
            start[cell_id] = time_val
            df.at[i,'ancestor'] = cell_id
            continue

        elif event in ["div", "fusion", "death", "out", "end"]:

            for cell_id in output:
                ancestor[cell_id] = ancestor[input[0]]

        # DBG: Missing info
        if input[0] not in ancestor:
            print("error")
            break

        df.at[i,'ancestor'] =  ancestor[input[0]]

    for i, row in df.iterrows():
        output = row["output"]
        
    return df
    
csv_radio_data_to_df(TEST_FILE)

normal_[n1]_0gy_201104.nd2u2510Gy1A.csv


,time,event,input,output,ancestor
0,0,begin,[],[C1],C1
1,66,div,[C1],"[C1.1, C1.2]",C1
2,238,div,[C1.1],"[C1.1.1, C1.1.2]",C1
3,386,div,[C1.1.1],"[C1.1.1.1, C1.1.1.2]",C1
4,432,end,[C1.1.1.1],[],C1
...,...,...,...,...,...
120,432,end,[C9.2.1.1],[],C9
121,432,end,[C9.2.1.2],[],C9
122,350,div,[C9.2.2],"[C9.2.2.1, C9.2.2.2]",C9
123,432,end,[C9.2.2.1],[],C9


In [5]:
def draw_trees(filepath):
    odd_day_color = "#F8F8FF"
    even_day_color = "#FFFAFA"
    df = csv_radio_data_to_df(filepath) #filepath # TODO: extract df from csv
    canvas_width = 1000
    canvas_height = 500
    left_offset = 7
    top_offset = 10
    day_height = (canvas_height-top_offset)/5 # TODO: adapt to number of days
    time_scale = (canvas_height-top_offset) / 720 #0.3
    tree_width = 200
    
    # Drawing canvas
    d = draw.Drawing(canvas_width, canvas_height, origin=(0,0), displayInline=False) # TODO: adapt to number of days and cell trees

    # Day subsection
    for day in range(0,5):
        color = odd_day_color
        if day % 2 == 0:
            color = even_day_color
        r = draw.Rectangle(left_offset, (canvas_height - day_height - day*day_height) - top_offset,canvas_width - left_offset, day_height, fill=color)
        r.appendTitle("Our first rectangle")  # Add a tooltip
        d.append(r)
        
    # Y graduation (time)
    for day, y in enumerate([0,144,288,432,576,720]):
        d.append(draw.Text(str(y), 4, 0, (canvas_height-top_offset)-(day*day_height), fill='black'))
        d.append(draw.Text("D"+str(day), 4, 0, (canvas_height-top_offset)-(day*day_height) - day_height/2, fill='red'))
        
    tree_width = (canvas_width - left_offset) / df["ancestor"].nunique()
    
    for tree_id, ancestor in enumerate(df["ancestor"].unique()):
        lineage = df[df["ancestor"] == ancestor].sort_values("time")
        #display(lineage)
        #print("Drawing lineage tree of "+ancestor)
        
        cell_position = dict()
        cell_sub_tree_width = dict()

        for index, row in lineage.iterrows():
            time = row["time"]
            event = row["event"]
            input_cells = row["input"]
            output_cells = row["output"]

            #print("Event:", time, event, input_cells, output_cells)

            if event == "begin":
                # Draw a cell with id label
                cell = output_cells[0]
                cell_position[cell] = (left_offset+tree_width/2, canvas_height-top_offset)
                cell_sub_tree_width[cell] = tree_width
                d.append(draw.Text(cell, 4, cell_position[cell][0]-2, cell_position[cell][1]+top_offset/2, fill='black'))
                d.append(draw.Circle(cell_position[cell][0], cell_position[cell][1], 1.5, fill='white', stroke_width=0.75, stroke='black'))

            if event == "div":
                # Draw vertical line to division time
                mother_cell = input_cells[0]
                mother_cell_x = cell_position[mother_cell][0]
                mother_cell_y = cell_position[mother_cell][1]
                children_cells_y = canvas_height-top_offset - time*time_scale
                d.append(draw.Text(str(time), 4, mother_cell_x+2, children_cells_y+2, fill='black'))
                d.append(draw.Line(mother_cell_x, 
                                   mother_cell_y-1.5, 
                                   mother_cell_x, 
                                   children_cells_y, 
                                   stroke='black', stroke_width=1, fill='none'))
                # draw division children cells nodes
                for output_id, cell in enumerate(output_cells):
                    cell_sub_tree_width[cell] = cell_sub_tree_width[mother_cell] / len(output_cells)
                    local_left_offset = mother_cell_x - cell_sub_tree_width[mother_cell]/2
                    cell_position[cell] = (local_left_offset+cell_sub_tree_width[cell]/2+cell_sub_tree_width[cell]*output_id, children_cells_y)
                    d.append(draw.Circle(cell_position[cell][0], cell_position[cell][1], 1.5, fill='white', stroke_width=0.75, stroke='black'))
                    #d.append(draw.Text(cell, 4, cell_position[cell][0]-len(cell), cell_position[cell][1]+5, fill='black'))

                # draw link between children of the division
                previous_cell = output_cells[0]
                for cell in output_cells[1:]:
                    d.append(draw.Line(cell_position[previous_cell][0]+1.5, 
                                   cell_position[previous_cell][1], 
                                   cell_position[cell][0]-1.5, 
                                   cell_position[cell][1], 
                                   stroke='black', stroke_width=1, fill='none'))
                    previous_cell = cell

            if event == "fusion":
                output_cell = output_cells[0]
                output_cell_x = [cell_position[cell][0] for cell in input_cells]
                output_cell_x = sum(output_cell_x) / len(output_cell_x)
                output_cell_y = canvas_height-top_offset - time*time_scale
                cell_sub_tree_width[output_cell] = sum([cell_sub_tree_width[i] for i in input_cells]) # inherit mothers width
                for input_cell in input_cells:
                    d.append(draw.Line(cell_position[input_cell][0], cell_position[input_cell][1]-1.5, output_cell_x, output_cell_y, stroke='black', stroke_width=1, fill='none'))
                cell_position[output_cell] = (output_cell_x, output_cell_y)
                d.append(draw.Circle(output_cell_x, output_cell_y, 1.5, fill='white', stroke_width=0.75, stroke='black'))
                d.append(draw.Text(str(time), 4, output_cell_x-3, output_cell_y+5, fill='black'))
                #d.append(draw.Text(cell, 4, output_cell_x-len(cell), output_cell_y+5, fill='black'))


            if event == "end":
                x_end = cell_position[input_cells[0]][0]
                y_end = canvas_height-top_offset - time*time_scale
                d.append(draw.Line(cell_position[input_cells[0]][0], cell_position[input_cells[0]][1]-1.5, x_end, y_end, stroke='black', stroke_width=1, fill='none'))
                d.append(draw.Circle(x_end, y_end, 1.5, fill='white', stroke_width=0.75, stroke='black'))
                d.append(draw.Text(str(time), 4, x_end-3, y_end-7, fill='black'))

            if event == "death":
                x_end = cell_position[input_cells[0]][0]
                y_end = canvas_height - top_offset - time*time_scale
                d.append(draw.Line(cell_position[input_cells[0]][0], cell_position[input_cells[0]][1]-1.5, x_end, y_end, stroke='black', stroke_width=1, fill='none'))
                d.append(draw.Line(x_end-2, y_end-2, x_end+2, y_end+2, stroke='red', stroke_width=1, fill='none'))
                d.append(draw.Line(x_end-2, y_end+2, x_end+2, y_end-2, stroke='red', stroke_width=1, fill='none'))
                d.append(draw.Text(str(time), 4, x_end-3, y_end-7, fill='black'))

            if event == "out":
                x_end = cell_position[input_cells[0]][0]
                y_end = canvas_height - top_offset - time*time_scale
                d.append(draw.Line(cell_position[input_cells[0]][0], cell_position[input_cells[0]][1]-1.5, x_end, y_end, stroke='black', stroke_width=1, fill='none'))
                d.append(draw.Line(x_end, y_end, x_end-3, y_end-4, stroke='blue', stroke_width=1, fill='none'))
                d.append(draw.Line(x_end+2, y_end, x_end-1, y_end-4, stroke='blue', stroke_width=1, fill='none'))
                d.append(draw.Text(str(time), 4, x_end-3, y_end-7, fill='black'))
            
        left_offset += tree_width
        #print(cell_position)
    d.setPixelScale(2)  # Set number of pixels per geometry unit
    return d  # Display as SVG
    print("finished")
    
draw_trees(TEST_FILE)

normal_[n1]_0gy_201104.nd2u2510Gy1A.csv
